In [1]:
%load_ext autoreload 
%autoreload 2

In [1]:
from collections import defaultdict
import itertools
import os
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from IPython.display import Markdown
from matminer.datasets import load_dataset
from pymatgen.core import Composition

from modnet.preprocessing import MODData
#from modnet.models import MODNetModel
from modnet.featurizers import MODFeaturizer
from modnet.featurizers.presets import DeBreuck2020Featurizer

os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
import modnet
modnet.__version__

'0.1.12'

In [4]:
class CompositionOnlyFeaturizer(MODFeaturizer):
    composition_featurizers = DeBreuck2020Featurizer.composition_featurizers
    
    def featurize_composition(self, df):
        """ Applies the preset composition featurizers to the input dataframe,
        renames some fields and cleans the output dataframe.

        """
        from pymatgen.core.periodic_table import Element 
        import numpy as np
        from modnet.featurizers import clean_df
        df = super().featurize_composition(df)
        _orbitals = {"s": 1, "p": 2, "d": 3, "f": 4}
        df['AtomicOrbitals|HOMO_character'] = df['AtomicOrbitals|HOMO_character'].map(_orbitals)
        df['AtomicOrbitals|LUMO_character'] = df['AtomicOrbitals|LUMO_character'].map(_orbitals)

        df['AtomicOrbitals|HOMO_element'] = df['AtomicOrbitals|HOMO_element'].apply(
            lambda x: -1 if not isinstance(x, str) else Element(x).Z
        )
        df['AtomicOrbitals|LUMO_element'] = df['AtomicOrbitals|LUMO_element'].apply(
            lambda x: -1 if not isinstance(x, str) else Element(x).Z
        )

        df = df.replace([np.inf, -np.inf, np.nan], 0)
        
        return clean_df(df)

class CompositionContainer:
    def __init__(self, composition):
        self.composition = composition

In [5]:

PRECOMPUTED_MODDATA = "./precomputed/expt_is_metal_benchmark_moddata.pkl.gz"

if os.path.isfile(PRECOMPUTED_MODDATA):
    data = MODData.load(PRECOMPUTED_MODDATA)
else:
    # Use a fresh copy of the dataset
    df = load_dataset("matbench_expt_is_metal")
    df["composition"] = df["composition"].map(Composition)
    df["structure"] = df["composition"].map(CompositionContainer)
    
    data = MODData(
        structures=df["structure"].tolist(), 
        targets=df["is_metal"].tolist(), 
        target_names=["is_metal"],
        featurizer=CompositionOnlyFeaturizer(n_jobs=8),
        num_classes = {'is_metal':2}
    )
    data.featurize()
    # As this is a small data/feature set, order all features 
    data.feature_selection(n=-1)

2023-02-06 14:56:50,006 - modnet - INFO - Loaded CompositionOnlyFeaturizer featurizer.
2023-02-06 14:56:50,010 - modnet - INFO - Computing features, this can take time...
2023-02-06 14:56:50,012 - modnet - INFO - Data has successfully been featurized!
2023-02-06 14:56:50,020 - modnet - INFO - Multiprocessing on 1 workers.
2023-02-06 14:56:50,021 - modnet - INFO - Computing "self" MI (i.e. information entropy) of features


0it [00:00, ?it/s]

2023-02-06 14:56:50,025 - modnet - INFO - Computing cross NMI between all features...



0it [00:00, ?it/s]

2023-02-06 14:56:50,065 - modnet - INFO - Starting target 1/1: is_metal ...
2023-02-06 14:56:50,066 - modnet - INFO - Computing mutual information between features and target...


ValueError: The input features DataFrame and the target variable DataFrame should contain the same number of data points.